In [6]:
import subprocess
import os
import os.path
import sys

In [7]:

def run_planner(domain, problem, goal_set, static_beta):
    cmd = ""
    if static_beta:
        cmd = ["ulimit -v 10000000; PFT.x" + " " + domain + " " + problem + " " + goal_set+ " -mcS"]
    else:
        cmd = ["ulimit -v 10000000; PFT.x" + " " + domain + " " + problem + " " + goal_set+ " -mcL"]
    #print(cmd)
    pipe = subprocess.Popen(
       cmd,
        stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)
    answer, stderr = pipe.communicate()
    #print(answer.decode())

    answerlist = answer.decode().split('\n')[2:]
    #print(answerlist)
    actionlist = answerlist[0].split(',')[:-1]
    timelist = answerlist[1].split(',')[:-1]
    fluentnumber = float(answerlist[2])
    Qbeta1 = float(answerlist[4])
    QbetaInf = float(answerlist[5])
    QVbeta = float(answerlist[6])
    Qlm = float(answerlist[7])
    delete_relaxed_cost = int(answerlist[8])
    
    return (actionlist, timelist, (fluentnumber, Qbeta1, QbetaInf, QVbeta, Qlm, delete_relaxed_cost))

def run_lama(domain, problem):
    cmd = [ "time", "python", "/home/jobasha/Documents/PhD/Planners/FastDownward/fast-downward.py",
             "--alias", "seq-sat-lama-2011", domain, problem]
    #print(cmd)
    pipe = subprocess.Popen(
        cmd,
        stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    answer, stderr = pipe.communicate()
    #print(answer.decode())
    #print(stderr.decode())
    time_taken = -1
    try:
        answerlist = stderr.decode().split("\n")
        answertemp1 = answerlist[0].split("system")
        answertemp2 = answertemp1[0].split("user")
        #print(answertemp2)
        time_taken = sum(list(map(float, answertemp2 )))
    except:
        take_taken = -1

    file = open("sas_plan.1","r")
    counter = 0
    plan = []
    for lines in file :
        counter = counter + 1
        plan.append(lines.strip().upper())

    plan = plan[:-1]
#     print("\t\tBetaS1...")
#     sys.stdout.flush()
    res_S_b1 = run_plan_rec_thresh(domain,problem,goal_set,"sas_plan.1", "-Sab 1")
#     print("\t\tBetaSInf...")
#     sys.stdout.flush()
    res_S_binf = run_plan_rec_thresh(domain,problem,goal_set,"sas_plan.1", "-Sanb 1000000000")
#     print("\t\tBetaV...")
#     sys.stdout.flush()
    res_V_b1 = run_plan_rec_thresh(domain,problem,goal_set,"sas_plan.1", "-ab 1")
#     print("\t\tLM...")
#     sys.stdout.flush()
    res_LM = run_plan_rec_thresh(domain,problem,goal_set,"sas_plan.1", "-l")
    
    return (plan, (time_taken, res_S_b1[0], res_S_binf[0], res_V_b1[0], res_LM[0] , res_S_b1[1]))

def run_plan_rec_thresh(domain,problem,goal_set,hyp_file,flags):
    try:
        exe = "ulimit -Sv 10000000; plan_recognition_threshold.x"
        files = domain + " " + problem + " " + goal_set + " " + hyp_file
        cmd = [exe + " " + files + " " + flags]
        pipe = subprocess.Popen(
           cmd,
            stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)
        answer, stderr = pipe.communicate()

        answerlist = answer.decode().split('\n')
        delete_relaxed_cost =  int(answerlist[2])
        Q= int(answerlist[1])
    except :
        Q = -1
        delete_relaxed_cost = -1
        print(answer.decode())
    return (Q,delete_relaxed_cost)

def SubDirPath (d):
    return list(filter(os.path.isdir, [os.path.join(d,f) for f in os.listdir(d)]))
    


In [ ]:
d = '/home/jobasha/Documents/Dropbox/Repositories/planningfortransparency/pddl/yar/'

subds = SubDirPath(d)

file = open(d +'results.csv', 'w+')

for sd in subds :
    domain = sd + "/domain.pddl"
    subsubds = SubDirPath(sd)
    domains_to_results = {}
    
    for ssd in subsubds :
        goal_set = ssd + "/hyps.dat"
        planning_dir = os.listdir(ssd + "/planning/")
        
        for problem in planning_dir :
            try:
                print("LAMA,\tProblem: " + problem)
                sys.stdout.flush()
                lama_results = run_lama(domain, ssd + "/planning/" + problem)
            except BaseException as e:
                print("\t\tLAMA fails, Problem: " + problem + " " + str(e))
                lama_results = ([], (-1, -1, -1, -1, -1 , -1))
            
                
            try:
                print("PFP_V,\tProblem: " + problem)
                sys.stdout.flush()
                pfp_v_results = run_planner(domain,ssd + "/planning/" + problem,goal_set, False)

                #domains_to_results[sd] = (lama_results, pfp_results)
            except BaseException as e:
                print("\t\tPFP_V fails, Problem: " + problem + " " + str(e))
                pfp_v_results = ([], [-1], (-1, -1, -1, -1, -1, -1))
                
#             try:
#                 print("PFP_S,\tProblem: " + problem)
#                 sys.stdout.flush()
#                 pfp_s_results = run_planner(domain,ssd + "/planning/" + problem,goal_set, True)

#                 #domains_to_results[sd] = (lama_results, pfp_results)
#             except BaseException as e:
#                 print("\t\tPFP_S fails, Problem: " + problem + " " + str(e))
#                 pfp_s_results = ([], [-1], (-1, -1, -1, -1, -1, -1))
                
            domain_name = sd.split('/')[-1]
            # Domain Name, Lama Time, LamaB1, LamaBinf, LamaVB, LamaLm, Delete, fluents, V_b1, V_binf, V_VB, V_lm, Delete, V_time, fluents, S_b1, S_binf, S_VB, S_lm, Delete, S_time,
            results_list = domain_name + "," + str(list(lama_results[1]) + list(pfp_v_results[2]) + [float(pfp_v_results[1][0])])
            print(results_list)
            file.write(results_list)
            file.write('\n')
            file.flush()


            
file.close()
            
            

            
#print(domains_to_results)
            
            



LAMA,	Problem: problem-p02-hyp_0.pddl
PFP_V,	Problem: problem-p02-hyp_0.pddl
kitchen,[-1, 0, 0, 0, 0, 19, 51.0, 2.0, 1.0, 2.0, 23.0, 19, 0.088394]
LAMA,	Problem: problem-p02-hyp_2.pddl
PFP_V,	Problem: problem-p02-hyp_2.pddl
kitchen,[-1, 0, 0, 0, 0, 5, 51.0, 11.0, 11.0, 11.0, 11.0, 5, 0.022125]
LAMA,	Problem: problem-p02-hyp_1.pddl
PFP_V,	Problem: problem-p02-hyp_1.pddl
kitchen,[-1, 0, 0, 0, 0, 6, 51.0, 8.0, 10.0, 10.0, 10.0, 6, 0.032207]
LAMA,	Problem: problem-p01-hyp_0.pddl
